# Data imputation

This dataset has been downloaded from  Kaggle https://www.kaggle.com/karangadiya/fifa19. License: [CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In this notebook we will do data processing for the dataset, imputing values that are missing based on present data.

## Step 1: Import libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, minmax_scale, scale

import matplotlib.pyplot as plt
import seaborn as sns
import bokeh as bk

## Step 2: Load data

First, we define where our data is and where we will store the imputated file

In [2]:
DATA = "../Data"
INPUT_FILE_NAME = f"{DATA}/FootballPlayerRawDataset.csv"

ATT_FILE_NAME = f"{DATA}/FootballPlayerPreparedCleanAttributes.csv"
IMPUTED_ATT_FILE_NAME = f"{DATA}/ImputedFootballPlayerPreparedCleanAttributes.csv"

ONE_HOT_ENCODED_CLASSES_FILE_NAME = f"{DATA}/FootballPlayerOneHotEncodedClasses.csv"
IMPUTED_ONE_HOT_ENCODED_CLASSES_FILE_NAME = f"{DATA}/ImputedFootballPlayerOneHotEncodedClasses.csv"

Now we load the data and show its info

In [3]:
dataset = pd.read_csv(INPUT_FILE_NAME, sep=",")

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18207 non-null  int64  
 1   ID                        18207 non-null  int64  
 2   Name                      18207 non-null  object 
 3   Age                       18207 non-null  int64  
 4   Photo                     18207 non-null  object 
 5   Nationality               18207 non-null  object 
 6   Flag                      18207 non-null  object 
 7   Overall                   18207 non-null  int64  
 8   Potential                 18207 non-null  int64  
 9   Club                      17966 non-null  object 
 10  Club Logo                 18207 non-null  object 
 11  Value                     18207 non-null  object 
 12  Wage                      18207 non-null  object 
 13  Special                   18207 non-null  int64  
 14  Prefer

## Step 3: Data cleaning

### Step 3.1: Remove unnecessary columns

We remove all goalkeepers as we will also remove the columns that represent their statistics

In [5]:
dataset.drop(dataset[dataset.Position=='GK'].index, inplace=True)

Then we remove unnecesary columns that we think won't affect the overall score of a player:
- Id
- Name
- Photo
- Nationality and Flag
- Team
- Club and Club Logo
- Preferred Foot
- Work Rate
- Body Type
- Real Face
- Position
- Jersey Number
- Joined
- Loaned From
- Contract Valid Until
- Height
- Weight
- From LS to RB
- From GKDiving to GKReflexes


In [6]:
dataset.drop(dataset.loc[:, 'Unnamed: 0':'Name'].columns, inplace=True, axis = 1)
dataset.drop(dataset.loc[:, 'Photo':'Flag'].columns, inplace=True, axis = 1)
dataset.drop(dataset.loc[:, 'Club':'Club Logo'].columns, inplace=True, axis = 1)
dataset.drop(dataset.loc[:, 'Preferred Foot':'Preferred Foot'].columns, inplace=True, axis = 1)
dataset.drop(dataset.loc[:, 'Work Rate':'RB'].columns, inplace=True, axis = 1)
dataset.drop(dataset.loc[:, 'GKDiving':'GKReflexes'].columns, inplace=True, axis = 1)



In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16182 entries, 0 to 18206
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       16182 non-null  int64  
 1   Overall                   16182 non-null  int64  
 2   Potential                 16182 non-null  int64  
 3   Value                     16182 non-null  object 
 4   Wage                      16182 non-null  object 
 5   Special                   16182 non-null  int64  
 6   International Reputation  16134 non-null  float64
 7   Weak Foot                 16134 non-null  float64
 8   Skill Moves               16134 non-null  float64
 9   Crossing                  16134 non-null  float64
 10  Finishing                 16134 non-null  float64
 11  HeadingAccuracy           16134 non-null  float64
 12  ShortPassing              16134 non-null  float64
 13  Volleys                   16134 non-null  float64
 14  Dribbl

### Step 3.2: Parse strings to floats

Now we need to take care of the columns whose DType is object

- Let's start by the **Value** column: We will remove the '€' , 'K' and 'M' characters and then change its type to float

In [8]:
dataset["Value"] = dataset["Value"].str.replace('€','')
dataset["Value"] = dataset["Value"].str.replace('M','')
dataset["Value"] = dataset["Value"].str.replace('K','')
dataset["Value"] = dataset["Value"].astype(float)

- Next column to take care of is **Wage** column: We will remove the '€' , 'K' and 'M' characters and then change its type to float

In [9]:
dataset["Wage"] = dataset["Wage"].str.replace('€','')
dataset["Wage"] = dataset["Wage"].str.replace('K','')
dataset["Wage"] = dataset["Wage"].astype(float)

- The last column that needs processing is **Release Clause**: We will remove the '€' , 'K' and 'M' characters and then change its type to float

In [10]:
dataset["Release Clause"] = dataset["Release Clause"].str.replace('€','')
dataset["Release Clause"] = dataset["Release Clause"].str.replace('M','')
dataset["Release Clause"] = dataset["Release Clause"].str.replace('K','')
dataset["Release Clause"] = dataset["Release Clause"].astype(float)

Finally let's check the dataset info after the changes

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16182 entries, 0 to 18206
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       16182 non-null  int64  
 1   Overall                   16182 non-null  int64  
 2   Potential                 16182 non-null  int64  
 3   Value                     16182 non-null  float64
 4   Wage                      16182 non-null  float64
 5   Special                   16182 non-null  int64  
 6   International Reputation  16134 non-null  float64
 7   Weak Foot                 16134 non-null  float64
 8   Skill Moves               16134 non-null  float64
 9   Crossing                  16134 non-null  float64
 10  Finishing                 16134 non-null  float64
 11  HeadingAccuracy           16134 non-null  float64
 12  ShortPassing              16134 non-null  float64
 13  Volleys                   16134 non-null  float64
 14  Dribbl

## Step 4: Data Imputation

In this section we will impute data based in the empty cells of our dataset

**First**, let's find the columns that have missing values.

In [12]:
def show_null_values(ds):
    null_dict = {attribute: ds[ds[attribute].isnull()].shape[0] for attribute in dataset.columns}
    return null_dict

In [33]:
null_dict = show_null_values(dataset)
null_dict

{'Age': 0,
 'Overall': 0,
 'Potential': 0,
 'Value': 0,
 'Wage': 0,
 'Special': 0,
 'International Reputation': 48,
 'Weak Foot': 48,
 'Skill Moves': 48,
 'Crossing': 48,
 'Finishing': 48,
 'HeadingAccuracy': 48,
 'ShortPassing': 48,
 'Volleys': 48,
 'Dribbling': 48,
 'Curve': 48,
 'FKAccuracy': 48,
 'LongPassing': 48,
 'BallControl': 48,
 'Acceleration': 48,
 'SprintSpeed': 48,
 'Agility': 48,
 'Reactions': 48,
 'Balance': 48,
 'ShotPower': 48,
 'Jumping': 48,
 'Stamina': 48,
 'Strength': 48,
 'LongShots': 48,
 'Aggression': 48,
 'Interceptions': 48,
 'Positioning': 48,
 'Vision': 48,
 'Penalties': 48,
 'Composure': 48,
 'Marking': 48,
 'StandingTackle': 48,
 'SlidingTackle': 48,
 'Release Clause': 1439}

As we can see there are 48 players with all their skill attributes as null, if we have a closer look at this subset all these players have an overall score of 62 so we can calculate the mean of the other players with 62 of overall score and impute the missing attributes

In [25]:
dataset[dataset['Vision'].isnull()].head(48)

,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Crossing,...,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause
13236,33,62,62,120.0,1.0,1510,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13237,29,62,62,300.0,1.0,1532,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13238,35,62,62,140.0,3.0,1573,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13239,20,62,72,425.0,1.0,1610,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13240,24,62,66,400.0,1.0,1481,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13241,21,62,72,425.0,1.0,1692,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13242,23,62,70,450.0,2.0,1663,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13243,19,62,78,600.0,5.0,1328,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13244,30,62,62,230.0,2.0,1244,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13245,21,62,69,425.0,1.0,1549,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To see the means of these attributes we can use describe from pandas

In [50]:
players_subset = dataset[dataset["Overall"] == 62]
subset_metrics = players_subset.describe()
subset_metrics

,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Crossing,...,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,Release Clause
count,798.000000,798.0,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,...,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,798.000000,738.000000
mean,23.895990,62.0,68.378446,391.635338,2.001253,1558.259398,1.002667,2.877333,2.262667,49.484000,...,54.501333,46.457333,50.020000,50.189333,47.913333,54.189333,48.121333,49.057333,47.245333,482.533913
std,4.215102,0.0,5.488634,133.228141,1.465196,113.140963,0.050027,0.563101,0.452589,11.378363,...,12.785632,16.274393,11.179537,9.981106,10.144957,6.189714,14.835948,17.052305,16.583346,277.157643
min,16.000000,62.0,62.000000,0.000000,0.000000,1141.000000,1.000000,1.000000,2.000000,20.000000,...,17.000000,11.000000,18.000000,12.000000,21.000000,36.000000,10.000000,11.000000,10.000000,1.000000
25%,21.000000,62.0,63.000000,300.000000,1.000000,1487.000000,1.000000,2.877333,2.000000,42.000000,...,46.000000,32.000000,45.000000,44.000000,41.000000,50.000000,37.000000,35.000000,34.000000,324.000000
50%,23.000000,62.0,68.000000,375.000000,2.000000,1564.500000,1.000000,3.000000,2.000000,52.000000,...,55.000000,54.000000,52.000000,51.000000,47.913333,54.189333,55.000000,58.000000,55.000000,508.000000
75%,27.000000,62.0,72.000000,475.000000,2.000000,1643.500000,1.000000,3.000000,2.262667,58.000000,...,64.000000,59.000000,58.000000,58.000000,55.000000,58.000000,59.000000,62.000000,60.000000,684.000000
max,38.000000,62.0,86.000000,700.000000,11.000000,1879.000000,2.000000,5.000000,4.000000,70.000000,...,90.000000,74.000000,69.000000,75.000000,80.000000,82.000000,76.000000,73.000000,71.000000,998.000000


Now we will get the names of the attributes that are null

In [41]:
null_attributes = [attr for attr in null_dict.keys() if null_dict[attr] > 0]
null_attributes

['International Reputation',
 'Weak Foot',
 'Skill Moves',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'Release Clause']

Finally we will replace all the missing cells with the mean of the corresponding attribute, we are using **mean imputation** because this is a very small subset and the bias of the data will not grow significantly

In [49]:
empty_rows_indexes = dataset[dataset['Vision'].isnull()].index.to_list()

for index in empty_rows_indexes:
    for attr in null_attributes:
        dataset.loc[index, attr] = subset_metrics.loc['mean', attr]
show_null_values(dataset)

{'Age': 0,
 'Overall': 0,
 'Potential': 0,
 'Value': 0,
 'Wage': 0,
 'Special': 0,
 'International Reputation': 0,
 'Weak Foot': 0,
 'Skill Moves': 0,
 'Crossing': 0,
 'Finishing': 0,
 'HeadingAccuracy': 0,
 'ShortPassing': 0,
 'Volleys': 0,
 'Dribbling': 0,
 'Curve': 0,
 'FKAccuracy': 0,
 'LongPassing': 0,
 'BallControl': 0,
 'Acceleration': 0,
 'SprintSpeed': 0,
 'Agility': 0,
 'Reactions': 0,
 'Balance': 0,
 'ShotPower': 0,
 'Jumping': 0,
 'Stamina': 0,
 'Strength': 0,
 'LongShots': 0,
 'Aggression': 0,
 'Interceptions': 0,
 'Positioning': 0,
 'Vision': 0,
 'Penalties': 0,
 'Composure': 0,
 'Marking': 0,
 'StandingTackle': 0,
 'SlidingTackle': 0,
 'Release Clause': 1391}

Next step is to impute the release clause value, so let's have a look at the metrics this field

In [51]:
dataset.describe()["Release Clause"]

count    14791.000000
mean       225.770775
std        301.537756
min          1.000000
25%          2.400000
50%         16.200000
75%        438.000000
max        999.000000
Name: Release Clause, dtype: float64

In [52]:
dataset.corr(numeric_only=True)["Release Clause"]

Age                        -0.041684
Overall                    -0.418272
Potential                  -0.406593
Value                       0.276573
Wage                       -0.216101
Special                    -0.338467
International Reputation   -0.168303
Weak Foot                  -0.079265
Skill Moves                -0.305151
Crossing                   -0.216301
Finishing                  -0.196934
HeadingAccuracy            -0.165998
ShortPassing               -0.318979
Volleys                    -0.215231
Dribbling                  -0.271523
Curve                      -0.235178
FKAccuracy                 -0.205295
LongPassing                -0.253433
BallControl                -0.336666
Acceleration               -0.116795
SprintSpeed                -0.126720
Agility                    -0.137033
Reactions                  -0.363540
Balance                    -0.064563
ShotPower                  -0.258734
Jumping                    -0.068497
Stamina                    -0.159770
S

There is no strong correlation of release clause with any of the other fields, so we can't a linear regression that is good enough, so we will use **hot deck imputation** to fill the empty cells. 
This imputation method gets rows with similar attributes and then choses a random value to copy.
We will be looking for players with similar *Overall* and *Potential* which are the two most correlated attributes with *Release Clause*

In [ ]:
def get_random_release_clause_value(dataset, overall_value, potential_value):
    min_overall_value = overall_value - 2
    max_overall_value = overall_value + 2
    
    min_potential_value = potential_value - 3
    max_potential_value = potential_value + 3
    subdataset = dataset[(min_overall_value <= dataset['Overall']) & 
                         (max_overall_value >= dataset['Overall']) &
                         (min_potential_value <= dataset['Potential']) &
                         (max_potential_value >= dataset['Potential'])]
    mean = subdataset['total_bedrooms'].mean()
    return mean